# Bibliotecas

In [1]:
import ranking
import salvar_resultados as sr
import imprimir_resultados as ir
import funcoes_bert as fb

import pandas as pd
import tensorflow as tf

import time

## Constantes

In [ ]:
SEED = 42

# Lendo os Arquivos

In [ ]:
#arquivos = ['ale_1_1']
#arquivos = ['ale_5_1']
#arquivos = ['hn_1_1']
arquivos = ['hn_5_1']

lista_df_treino = []
lista_df_val = []
lista_df_teste = []
for arquivo in arquivos:

    df = pd.read_csv(f"Dados/Datasets/Treino-Validação/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df)

    df = pd.read_csv(f"Dados/Datasets/Treino-Validação/Validação/{arquivo}_validação.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_val.append(df)

    df = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_teste.append(df)

In [ ]:
#lista_df_val[0].dtypes

In [ ]:
#print(f"Tamanho do Dataset de Treino 1 por 1:\n\n\t| Ale\t| Hard\t|\n1 por 1\t| {lista_df_treino[0].shape[0]}\t| {lista_df_treino[2].shape[0]}\t|\n5 por 1\t| {lista_df_treino[1].shape[0]}\t| {lista_df_treino[3].shape[0]}\t|")
#print(f"\nTamanho do Dataset de Validação 1 por 1:\n\n\t| Ale\t| Hard\t|\n1 por 1\t| {lista_df_val[0].shape[0]}\t| {lista_df_val[2].shape[0]}\t|\n5 por 1\t| {lista_df_val[1].shape[0]}\t| {lista_df_val[3].shape[0]}\t|")
#print(f"\nTamanho do Dataset de Teste 1 por 1:\n\n\t| Ale\t| Hard\t|\n1 por 1\t| {lista_df_teste[0].shape[0]}\t| {lista_df_teste[2].shape[0]}\t|\n5 por 1\t| {lista_df_teste[1].shape[0]}\t| {lista_df_teste[3].shape[0]}\t|")


# BERT

In [ ]:
def salvar_modelo_bert(modelo, nome):
    
    nome_arquivo = f"Dados/Modelos/BERT/{nome}/"
    modelo.save_pretrained(nome_arquivo)

In [ ]:
def criar_dicionario(indice, titulo, ean, categoria, colunas):
    
    return {
            colunas[0] : indice, colunas[1] : titulo, colunas[2] : ean, colunas[3] : categoria
           }


def criar_df_match(df_concat, ean_repetido, coluna_saida = 'titulo_sa'):

    COLUNAS = ("indice", coluna_saida, "ean", "categoria")

    df_matches = pd.DataFrame(columns = COLUNAS)
    for ean in ean_repetido:

        # pega o indice da primeira linha com aquele EAN
        filtro = (df_concat['ean'] == ean)
        indice = next(iter(filtro.index[filtro]))

        dicionario = criar_dicionario(
                                    indice = indice,
                                    titulo = df_concat.loc[indice][coluna_saida],
                                    ean = df_concat.loc[indice]['ean'],
                                    categoria = df_concat.loc[indice]['categoria'],
                                    colunas = COLUNAS
                                    )

        df_matches = df_matches.append(dicionario, ignore_index = True)

    df_matches.sort_values('indice', inplace = True)
    df_matches.reset_index(drop = True, inplace = True)

    return df_matches


## Rodando o BERT

In [ ]:
dir_metricas = "Dados/Resultados/Ranqueado/BERT"

In [ ]:
lista_df_resultado = []
#tam = 51
for nome, df_treino, df_val, df_teste in zip(arquivos, lista_df_treino, lista_df_val, lista_df_teste):

    '''df_teste = df_teste[:tam]
    df_treino = df_teste[:tam]
    df_val = df_teste[:tam]'''
    
    df_concat, df_matches = ranking.criar_df_teste(df_teste, "titulo_1", "titulo_2", 'titulo')

    X_treino = df_treino[["titulo_1", "titulo_2"]]
    X_val = df_val[["titulo_1", "titulo_2"]]
    X_teste = df_concat["titulo"]

    y_treino = df_treino["match"].to_list()
    y_val = df_val["match"].to_list()
    #y_teste = df_teste["match"].to_list()

    inicio_tempo = time.time()

    #(modelo, historico, embedding) = fb.pipeline_bert(X_treino, y_treino, X_val, y_val, X_teste)
    embedding = fb.pipeline_bert(X_teste)
    resultado = ranking.calcular_dis_2_vetores_cond(embedding, df_matches)
    
    final_tempo = time.time()
    
    # adicionando o tempo que demorou para o algoritmo rodar
    ranking.calcular_tempo(df_matches, inicio_tempo, final_tempo)

    # calcular acuracia_k e match rank
    indices, valores = ranking.calcular_metricas(df_matches, df_concat, resultado)

    #pd.DataFrame.from_dict(historico.history).to_csv(f'Dados/Resultados/Ranqueado/BERT/Histórico/{nome}_historico.csv', index = False)

    sr.salvar_resultado(nome, df_matches, dir_metricas)
    sr.salvar_resultado_categoria(nome, df_matches, dir_metricas)

    #salvar_modelo_bert(modelo, nome)

    break

In [ ]:
    a

# Resultados

## Geral

In [9]:
metodo = "BERT"
lista_resultado_bert = ir.retornar_resultados(metodo)

### Aleatório

#### 1:1

In [10]:
lista_resultado_bert[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,458.000000,458.000000,458.000000,458.000000,458.000000
mean,0.170306,0.434498,0.639738,119.906114,0.256782
std,0.376312,0.496233,0.480601,232.156737,0.362478
min,0.000000,0.000000,0.000000,1.000000,0.000571
25%,0.000000,0.000000,0.000000,3.000000,0.009055
50%,0.000000,0.000000,1.000000,19.000000,0.052632
75%,0.000000,1.000000,1.000000,110.500000,0.333333
max,1.000000,1.000000,1.000000,1750.000000,1.000000


#### 5:1

In [5]:
lista_resultado_bert[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,631.000000,631.000000,631.000000,631.000000,631.000000
mean,0.128368,0.402536,0.591125,220.434231,0.218341
std,0.334764,0.490798,0.492016,483.603135,0.332129
min,0.000000,0.000000,0.000000,1.000000,0.000265
25%,0.000000,0.000000,0.000000,4.000000,0.006557
50%,0.000000,0.000000,1.000000,21.000000,0.047619
75%,0.000000,1.000000,1.000000,152.500000,0.250000
max,1.000000,1.000000,1.000000,3774.000000,1.000000


### Hard Negative

#### 1:1

In [6]:
lista_resultado_bert[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,530.000000,530.000000,530.000000,530.000000,530.000000
mean,0.141509,0.420755,0.656604,112.143396,0.236208
std,0.348876,0.494147,0.475291,228.965980,0.340333
min,0.000000,0.000000,0.000000,1.000000,0.000617
25%,0.000000,0.000000,0.000000,3.000000,0.010444
50%,0.000000,0.000000,1.000000,16.000000,0.062500
75%,0.000000,1.000000,1.000000,95.750000,0.333333
max,1.000000,1.000000,1.000000,1620.000000,1.000000


#### 5:1

In [7]:
lista_resultado_bert[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,756.000000,756.000000,756.000000,756.000000,756.000000
mean,0.113757,0.411376,0.596561,185.373016,0.209162
std,0.317726,0.492409,0.490912,401.390597,0.317125
min,0.000000,0.000000,0.000000,1.000000,0.000345
25%,0.000000,0.000000,0.000000,4.000000,0.006791
50%,0.000000,0.000000,1.000000,23.000000,0.043478
75%,0.000000,1.000000,1.000000,147.250000,0.250000
max,1.000000,1.000000,1.000000,2899.000000,1.000000


## Por Categoria

### Celulares

In [11]:
metodo = "BERT/celulares"
lista_resultado_bert_cell = ir.retornar_resultados(metodo)

In [12]:
lista_resultado_bert_cell[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,83.000000,83.000000,83.000000,83.000000,83.000000
mean,0.096386,0.253012,0.457831,193.313253,0.157565
std,0.296913,0.437381,0.501247,303.386694,0.302900
min,0.000000,0.000000,0.000000,1.000000,0.000571
25%,0.000000,0.000000,0.000000,10.000000,0.003897
50%,0.000000,0.000000,0.000000,63.000000,0.015873
75%,0.000000,0.500000,1.000000,259.000000,0.101010
max,1.000000,1.000000,1.000000,1750.000000,1.000000


In [13]:
lista_resultado_bert_cell[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,88.000000,88.000000,88.000000,88.000000,88.000000
mean,0.045455,0.204545,0.363636,386.693182,0.096691
std,0.209493,0.405681,0.483802,595.351405,0.225009
min,0.000000,0.000000,0.000000,1.000000,0.000347
25%,0.000000,0.000000,0.000000,14.500000,0.002443
50%,0.000000,0.000000,0.000000,137.500000,0.007273
75%,0.000000,0.000000,1.000000,409.250000,0.069231
max,1.000000,1.000000,1.000000,2879.000000,1.000000


In [14]:
lista_resultado_bert_cell[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,83.000000,83.000000,83.000000,83.000000,83.000000
mean,0.132530,0.277108,0.493976,160.289157,0.194577
std,0.341127,0.450291,0.503003,234.941432,0.341047
min,0.000000,0.000000,0.000000,1.000000,0.000813
25%,0.000000,0.000000,0.000000,8.000000,0.005052
50%,0.000000,0.000000,0.000000,51.000000,0.019608
75%,0.000000,1.000000,1.000000,198.000000,0.126984
max,1.000000,1.000000,1.000000,1230.000000,1.000000


In [15]:
lista_resultado_bert_cell[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,121.000000,121.000000,121.000000,121.000000,121.000000
mean,0.082645,0.280992,0.429752,274.537190,0.149421
std,0.276489,0.451352,0.497099,396.352832,0.287175
min,0.000000,0.000000,0.000000,1.000000,0.000482
25%,0.000000,0.000000,0.000000,8.000000,0.002874
50%,0.000000,0.000000,0.000000,112.000000,0.008929
75%,0.000000,1.000000,1.000000,348.000000,0.125000
max,1.000000,1.000000,1.000000,2076.000000,1.000000


### Fogões

In [16]:
metodo = "BERT/fogoes"
lista_resultado_bert_fogoes = ir.retornar_resultados(metodo)

In [17]:
lista_resultado_bert_fogoes[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,148.000000,148.000000,148.000000,148.000000,148.000000
mean,0.148649,0.412162,0.655405,133.040541,0.226642
std,0.356950,0.493895,0.476850,263.067175,0.344687
min,0.000000,0.000000,0.000000,1.000000,0.000770
25%,0.000000,0.000000,0.000000,4.000000,0.010283
50%,0.000000,0.000000,1.000000,25.000000,0.040000
75%,0.000000,1.000000,1.000000,97.250000,0.250000
max,1.000000,1.000000,1.000000,1298.000000,1.000000


In [18]:
lista_resultado_bert_fogoes[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,220.000000,220.000000,220.000000,220.000000,220.000000
mean,0.113636,0.354545,0.577273,220.395455,0.193299
std,0.318093,0.479466,0.495119,499.910406,0.317678
min,0.000000,0.000000,0.000000,1.000000,0.000326
25%,0.000000,0.000000,0.000000,5.000000,0.007694
50%,0.000000,0.000000,1.000000,24.500000,0.040833
75%,0.000000,1.000000,1.000000,130.000000,0.200000
max,1.000000,1.000000,1.000000,3066.000000,1.000000


In [19]:
lista_resultado_bert_fogoes[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,176.000000,176.000000,176.000000,176.000000,176.000000
mean,0.107955,0.380682,0.659091,137.738636,0.193998
std,0.311208,0.486940,0.475367,273.014578,0.307322
min,0.000000,0.000000,0.000000,1.000000,0.000801
25%,0.000000,0.000000,0.000000,4.000000,0.010159
50%,0.000000,0.000000,1.000000,20.000000,0.050000
75%,0.000000,1.000000,1.000000,98.500000,0.250000
max,1.000000,1.000000,1.000000,1248.000000,1.000000


In [20]:
lista_resultado_bert_fogoes[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,228.000000,228.000000,228.000000,228.000000,228.000000
mean,0.092105,0.346491,0.561404,192.868421,0.176353
std,0.289811,0.476899,0.497307,427.499543,0.295128
min,0.000000,0.000000,0.000000,1.000000,0.000345
25%,0.000000,0.000000,0.000000,5.000000,0.008048
50%,0.000000,0.000000,1.000000,29.500000,0.033908
75%,0.000000,1.000000,1.000000,124.250000,0.200000
max,1.000000,1.000000,1.000000,2899.000000,1.000000


### Geladeiras

In [21]:
metodo = "BERT/geladeiras"
lista_resultado_bert_gel = ir.retornar_resultados(metodo)

In [22]:
lista_resultado_bert_gel[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,67.000000,67.000000,67.000000,67.000000,67.000000
mean,0.179104,0.597015,0.820896,39.776119,0.325249
std,0.386334,0.494200,0.386334,88.258543,0.357541
min,0.000000,0.000000,0.000000,1.000000,0.002132
25%,0.000000,0.000000,1.000000,2.000000,0.038519
50%,0.000000,1.000000,1.000000,7.000000,0.142857
75%,0.000000,1.000000,1.000000,26.000000,0.500000
max,1.000000,1.000000,1.000000,469.000000,1.000000


In [23]:
lista_resultado_bert_gel[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,96.000000,96.000000,96.000000,96.000000,96.000000
mean,0.145833,0.541667,0.750000,76.572917,0.274473
std,0.354792,0.500876,0.435286,184.574774,0.341824
min,0.000000,0.000000,0.000000,1.000000,0.000926
25%,0.000000,0.000000,0.750000,2.000000,0.019808
50%,0.000000,1.000000,1.000000,8.500000,0.118056
75%,0.000000,1.000000,1.000000,50.500000,0.500000
max,1.000000,1.000000,1.000000,1080.000000,1.000000


In [24]:
lista_resultado_bert_gel[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,71.000000,71.000000,71.000000,71.000000,71.000000
mean,0.169014,0.521127,0.760563,46.098592,0.289481
std,0.377432,0.503109,0.429777,80.298400,0.354833
min,0.000000,0.000000,0.000000,1.000000,0.002857
25%,0.000000,0.000000,1.000000,2.500000,0.025790
50%,0.000000,1.000000,1.000000,9.000000,0.111111
75%,0.000000,1.000000,1.000000,40.000000,0.416667
max,1.000000,1.000000,1.000000,350.000000,1.000000


In [25]:
lista_resultado_bert_gel[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,108.000000,108.000000,108.000000,108.000000,108.000000
mean,0.157407,0.490741,0.768519,50.518519,0.258649
std,0.365882,0.502245,0.423746,103.130756,0.346528
min,0.000000,0.000000,0.000000,1.000000,0.001534
25%,0.000000,0.000000,1.000000,3.000000,0.021059
50%,0.000000,0.000000,1.000000,12.000000,0.083333
75%,0.000000,1.000000,1.000000,47.500000,0.333333
max,1.000000,1.000000,1.000000,652.000000,1.000000


### Notebooks

In [26]:
metodo = "BERT/notebooks"
lista_resultado_bert_note = ir.retornar_resultados(metodo)

In [27]:
lista_resultado_bert_note[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,88.000000,88.000000,88.000000,88.000000,88.000000
mean,0.284091,0.534091,0.636364,100.306818,0.360553
std,0.453565,0.501695,0.483802,165.933192,0.424613
min,0.000000,0.000000,0.000000,1.000000,0.001307
25%,0.000000,0.000000,0.000000,1.000000,0.007917
50%,0.000000,1.000000,1.000000,7.000000,0.142857
75%,1.000000,1.000000,1.000000,127.500000,1.000000
max,1.000000,1.000000,1.000000,765.000000,1.000000


In [28]:
lista_resultado_bert_note[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,109.000000,109.000000,109.000000,109.000000,109.000000
mean,0.229358,0.522936,0.633028,195.247706,0.331474
std,0.422362,0.501781,0.484205,420.052447,0.399433
min,0.000000,0.000000,0.000000,1.000000,0.000489
25%,0.000000,0.000000,0.000000,2.000000,0.006623
50%,0.000000,1.000000,1.000000,9.000000,0.111111
75%,0.000000,1.000000,1.000000,151.000000,0.500000
max,1.000000,1.000000,1.000000,2047.000000,1.000000


In [29]:
lista_resultado_bert_note[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,116.000000,116.000000,116.000000,116.000000,116.000000
mean,0.181034,0.508621,0.689655,94.844828,0.289515
std,0.386717,0.502095,0.464642,238.241551,0.368079
min,0.000000,0.000000,0.000000,1.000000,0.000617
25%,0.000000,0.000000,0.000000,2.000000,0.014765
50%,0.000000,1.000000,1.000000,9.500000,0.105556
75%,0.000000,1.000000,1.000000,67.750000,0.500000
max,1.000000,1.000000,1.000000,1620.000000,1.000000


In [30]:
lista_resultado_bert_note[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,170.000000,170.000000,170.000000,170.000000,170.000000
mean,0.158824,0.488235,0.629412,199.605882,0.266715
std,0.366591,0.501338,0.484389,454.783885,0.354650
min,0.000000,0.000000,0.000000,1.000000,0.000371
25%,0.000000,0.000000,0.000000,2.250000,0.005473
50%,0.000000,0.000000,1.000000,11.500000,0.087121
75%,0.000000,1.000000,1.000000,182.750000,0.458333
max,1.000000,1.000000,1.000000,2699.000000,1.000000


### TVs

In [36]:
metodo = "BERT/tvs"
lista_resultado_bert_tvs = ir.retornar_resultados(metodo)

In [37]:
lista_resultado_bert_tvs[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,72.000000,72.000000,72.000000,72.000000,72.000000
mean,0.152778,0.416667,0.652778,106.805556,0.242568
std,0.362298,0.496466,0.479428,207.313985,0.350176
min,0.000000,0.000000,0.000000,1.000000,0.000826
25%,0.000000,0.000000,0.000000,3.000000,0.012821
50%,0.000000,0.000000,1.000000,20.500000,0.048810
75%,0.000000,1.000000,1.000000,80.000000,0.333333
max,1.000000,1.000000,1.000000,1211.000000,1.000000


In [33]:
lista_resultado_bert_tvs[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,118.000000,118.000000,118.000000,118.000000,118.000000
mean,0.110169,0.415254,0.618644,236.822034,0.205582
std,0.314436,0.494867,0.487791,546.627932,0.313528
min,0.000000,0.000000,0.000000,1.000000,0.000265
25%,0.000000,0.000000,0.000000,4.000000,0.007605
50%,0.000000,0.000000,1.000000,21.000000,0.047727
75%,0.000000,1.000000,1.000000,131.500000,0.250000
max,1.000000,1.000000,1.000000,3774.000000,1.000000


In [34]:
lista_resultado_bert_tvs[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,84.000000,84.000000,84.000000,84.000000,84.000000
mean,0.142857,0.440476,0.678571,90.654762,0.247138
std,0.352029,0.499426,0.469830,173.321798,0.343859
min,0.000000,0.000000,0.000000,1.000000,0.000956
25%,0.000000,0.000000,0.000000,3.000000,0.013115
50%,0.000000,0.000000,1.000000,19.500000,0.051587
75%,0.000000,1.000000,1.000000,76.250000,0.333333
max,1.000000,1.000000,1.000000,1046.000000,1.000000


In [35]:
lista_resultado_bert_tvs[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,129.000000,129.000000,129.000000,129.000000,129.000000
mean,0.085271,0.480620,0.627907,182.635659,0.205909
std,0.280374,0.501572,0.485247,416.814893,0.288443
min,0.000000,0.000000,0.000000,1.000000,0.000387
25%,0.000000,0.000000,0.000000,3.000000,0.008929
50%,0.000000,0.000000,1.000000,12.000000,0.083333
75%,0.000000,1.000000,1.000000,112.000000,0.333333
max,1.000000,1.000000,1.000000,2586.000000,1.000000
